In [1]:
import transformers
import torch
from time import perf_counter
import dotenv
import os

dotenv.load_dotenv()

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, token=os.getenv('HF_TOKEN'))
model = transformers.AutoModelForCausalLM.from_pretrained(model_id, token=os.getenv('HF_TOKEN')).to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Set pad token id if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.45it/s]


In [19]:
input_text = "testing, testing, 1, 2, "

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Move input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Set pad token id if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Generate text token by token
generated = inputs['input_ids']
attention_mask = inputs['attention_mask']

for _ in range(50):  # Generate 50 new tokens
    outputs = model.generate(
        generated,
        attention_mask=attention_mask,
        max_new_tokens=1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    new_token = outputs[0, -1].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1)
    generated = torch.cat([generated, new_token], dim=-1)
    attention_mask = torch.cat([attention_mask, torch.ones((1, 1), dtype=torch.long, device=device)], dim=-1)
    print(tokenizer.decode(new_token.squeeze().cpu()), end='', flush=True)

print("\n\nFull generated text:")
print(tokenizer.decode(generated[0].cpu(), skip_special_tokens=True))

3...
Welcome to the new and improved blog of The Art of Living! I'm excited to share my thoughts, experiences, and insights with you all on this new platform. I hope you'll join me on this journey and share your own stories and

Full generated text:
testing, testing, 1, 2, 3...
Welcome to the new and improved blog of The Art of Living! I'm excited to share my thoughts, experiences, and insights with you all on this new platform. I hope you'll join me on this journey and share your own stories and
